In [1]:
%%capture
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [9]:
import sys
sys.path.append("../")

import re,time,json,pickle
from collections import defaultdict
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn

from transformers import (BertTokenizer,BertConfig,BertModel)
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer, Anchi_Decoder,Anchi_Transformer
from model.fusionDataset import FusionDataset
from utils.generate_couplet import greedy_decode,beam_search_decode

config = BertConfig.from_pretrained('../AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('../AnchiBERT')
Anchibert = BertModel.from_pretrained('../AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at ../AnchiBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Load Necessary preproceeded Data

In [3]:
with open('../pretrained_model/char_map.json','r') as f:
    ix2glyph = defaultdict(lambda : '_')
    ix2glyph[0] = '[PAD]'
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    for i, k in enumerate(json.load(f).keys(),2):
        glyph2ix[k] = i
        ix2glyph[i] = k
with open('../pretrained_model/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    for i,k in enumerate(json.load(f).keys(),2):
        pinyin2ix[k] = i
with open('../pretrained_model/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

# Decoder Section

In [5]:
with open("../couplet/test/in.txt",encoding='utf8') as f:
    te_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("../couplet/test/out.txt",encoding='utf8') as f:
    te_out = [row.strip().split() for row in f.readlines()]

In [11]:
config = { # for Fusion_Anchi_Trans_Decoder
        'max_position_embeddings':50,
        'hidden_size':768,
        'font_weight_path':'../pretrained_model/glyph_weight.npy',
        'pinyin_embed_dim':30, # trainable
        'pinyin_path':'../pretrained_model/pinyin_map.json',
        'tag_size':30,
        'tag_emb_dim':10, # trainable
        'layer_norm_eps':1e-12,
        'hidden_dropout':0.1,
        'nhead':12,
        'num_layers':6 , #6, trainable
        'output_dim':9110,# fixed use glyph dim as output
        'device':device,
    }


# <model_name>_<optim>_<batch_num>_<lr>_<epoch>_<pinyin_embed_dim>_<tag_emb_dim>_<encoder layer>_<decoder layer>_<train_data_size>
name = 'fu_anchi_de_Adam_128_0001_60_6_30_10_51k'
model = Fusion_Anchi_Trans_Decoder(config)
model.load_state_dict(torch.load(f'../result/{name}.pt'))

<All keys matched successfully>

In [12]:
idx = 10

predict = greedy_decode(model=model,
                      bert=Anchibert,
                      tokenizer=tokenizer,
                      sent=te_in[idx],
                      glyph2ix=glyph2ix,
                      pinyin2ix=pinyin2ix,
                      pos2ix=pos2ix,
                      ix2glyph=ix2glyph,
                        device=device)

Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 26.53it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, ?it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 558.20it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 557.60it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 557.90it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 557.90it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 372.00it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 558.05it/s]


In [13]:
' '.join(te_in[idx]),' '.join(predict), ' '.join(te_out[idx])

('岁 尽 方 知 时 日 快', '春 来 不 觉 故 乡 多', '钱 空 始 觉 世 情 薄')

In [51]:
# idxs = list(range(20))
idxs = [4,20,13,17]
preds = []
for i, idx in enumerate(idxs):
    data = te_in[idx]

    predict = beam_search_decode(model=model,
                                k=5,
                              bert=Anchibert,
                              tokenizer=tokenizer,
                              sent=data,
                              glyph2ix=glyph2ix,
                              pinyin2ix=pinyin2ix,
                              pos2ix=pos2ix,
                              ix2glyph=ix2glyph,
                                device=device)[0][0]
    preds.append(predict)
    
    


Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 454.37it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, ?it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 518.01it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 1035.89it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 1036.91it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 517.88it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 517.94it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 518.01it/s]
Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
1it [00:00, 1035.89

In [52]:
for i, pred in enumerate(preds):
    print(f'sample {i}')
    print('top line: '+''.join(te_in[idxs[i]]))
    print('predict: '+ ''.join(pred))
    print('gold: ' + ''.join(te_out[idxs[i]]))
    print("================================")

sample 0
top line: 一句相思吟岁月
predict: 三分寂寞醉春秋
gold: 千杯美酒醉风情
sample 1
top line: 与地增春，其熙长趣
predict: 同天焕彩，此日今年
gold: 运躬足岁，我乐于心
sample 2
top line: 无花无酒无花酒
predict: 有酒有茶有墨香
gold: 有洞有天有洞天
sample 3
top line: 路边鸡放哨
predict: 天下鸟鸣春
gold: 野外虎巡山
